## Import Libraries

In [1]:
import pandas as pd 
import numpy as np 
import pickle
from skmultilearn.model_selection import iterative_train_test_split
from sklearn.model_selection import train_test_split
from sklearn.metrics import log_loss
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from skmultilearn.problem_transform import ClassifierChain
import tensorflow as tf
from tensorflow.keras.layers import Dense, Activation, Input, BatchNormalization

2024-03-07 23:49:27.265389: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-03-07 23:49:27.367076: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-07 23:49:27.367091: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-03-07 23:49:27.909400: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:
train = pd.read_csv('/home/abdelmoneim/سطح المكتب/for graduation project/dataset/train_features.csv')
test = pd.read_csv('/home/abdelmoneim/سطح المكتب/for graduation project/dataset/test_features.csv')
scored_target =pd.read_csv('/home/abdelmoneim/سطح المكتب/for graduation project/dataset/train_targets_scored.csv')

In [3]:
# list of name of moa and remove sig_id
target_cols = [col for col in scored_target.columns if col not in ['sig_id']]

# separating cell features and gene features
gene_features = []
cell_features = []
for i in train.columns:
    if i.startswith('g-'):
        gene_features.append(i)
    if i.startswith('c-'):
        cell_features.append(i)

print('the length of gene features', len(gene_features))
print('the lenght of cell features', len(cell_features))

# marge train inputs and target
train_merge = pd.merge(train, scored_target, on = 'sig_id', how = 'left')
print('the length of all inputs and target dataset', train_merge.shape)

the length of gene features 772
the lenght of cell features 100
the length of all inputs and target dataset (23814, 1082)


In [4]:
# encode categorical features in the train and test data
le = LabelEncoder()

train_merge['cp_type'] = le.fit_transform(train_merge['cp_type'])
train_merge['cp_time'] = le.fit_transform(train_merge['cp_time'])
train_merge['cp_dose'] = le.fit_transform(train_merge['cp_dose'])

test['cp_type'] = le.fit_transform(test['cp_type'])
test['cp_time'] = le.fit_transform(test['cp_time'])
test['cp_dose'] = le.fit_transform(test['cp_dose'])

In [5]:
x = train_merge.copy()
x.head()

,sig_id,cp_type,cp_time,cp_dose,g-0,g-1,g-2,g-3,g-4,g-5,...,tropomyosin_receptor_kinase_inhibitor,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor
0,id_000644bb2,1,0,0,1.0620,0.5577,-0.2479,-0.6208,-0.1944,-1.0120,...,0,0,0,0,0,0,0,0,0,0
1,id_000779bfc,1,2,0,0.0743,0.4087,0.2991,0.0604,1.0190,0.5207,...,0,0,0,0,0,0,0,0,0,0
2,id_000a6266a,1,1,0,0.6280,0.5817,1.5540,-0.0764,-0.0323,1.2390,...,0,0,0,0,0,0,0,0,0,0
3,id_0015fd391,1,1,0,-0.5138,-0.2491,-0.2656,0.5288,4.0620,-0.8095,...,0,0,0,0,0,0,0,0,0,0
4,id_001626bd3,1,2,1,-0.3254,-0.4009,0.9700,0.6919,1.4180,-0.8244,...,0,0,0,0,0,0,0,0,0,0


In [6]:
y = x[target_cols]

In [7]:
x.shape, test.shape, y.shape

((23814, 1082), (3982, 876), (23814, 206))

## gene features - AutoEncoder

In [8]:
# encoder part
inputs = Input(shape = (772,))
encoder_1 = Dense(512, activation = 'relu')(inputs)
batch_norm = BatchNormalization()(encoder_1)
encoder_2 = Dense(420, activation = 'relu')(batch_norm)

# decoder part
decodrer_1 = Dense(420, activation = 'relu')(encoder_2)
batch_norm = BatchNormalization()(decodrer_1)
decodrer_2 = Dense(512, activation = 'relu')(batch_norm)
batch_norm = BatchNormalization()(decodrer_2)
decoder_3 = Dense(772)(encoder_2)

model = tf.keras.Model(inputs = inputs, outputs = decoder_3)

model.compile(optimizer = 'adam', loss = 'mse')

In [9]:
with tf.device('/GPU:0'):
    model.fit(x[gene_features], x[gene_features], batch_size = 1000, epochs = 200)

Epoch 1/200


I0000 00:00:1708886687.766636      67 device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


24/24 [==============================] - 3s 9ms/step - loss: 1.0899
Epoch 2/200
24/24 [==============================] - 0s 7ms/step - loss: 0.7397
Epoch 3/200
24/24 [==============================] - 0s 7ms/step - loss: 0.6492
Epoch 4/200
24/24 [==============================] - 0s 7ms/step - loss: 0.5931
Epoch 5/200
24/24 [==============================] - 0s 6ms/step - loss: 0.5503
Epoch 6/200
24/24 [==============================] - 0s 6ms/step - loss: 0.5195
Epoch 7/200
24/24 [==============================] - 0s 6ms/step - loss: 0.4854
Epoch 8/200
24/24 [==============================] - 0s 6ms/step - loss: 0.4590
Epoch 9/200
24/24 [==============================] - 0s 6ms/step - loss: 0.4294
Epoch 10/200
24/24 [==============================] - 0s 6ms/step - loss: 0.3995
Epoch 11/200
24/24 [==============================] - 0s 6ms/step - loss: 0.3719
Epoch 12/200
24/24 [==============================] - 0s 6ms/step - loss: 0.3487
Epoch 13/200
24/24 [=============================

In [10]:
# take the output of the encoded part
encoder = tf.keras.Model(inputs = inputs, outputs = encoder_2)
encoder.save('encoder_gene_features.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## cell features - AutoEncoder

In [11]:
# encoder part
inputs = Input(shape = (100,))
encoder_1 = Dense(90, activation = 'relu')(inputs)
batch_norm = BatchNormalization()(encoder_1)
encoder_2 = Dense(75, activation = 'relu')(batch_norm)

# decoder part
decodrer_1 = Dense(75, activation = 'relu')(encoder_2)
batch_norm = BatchNormalization()(decodrer_1)
decodrer_2 = Dense(90, activation = 'relu')(batch_norm)
batch_norm = BatchNormalization()(decodrer_2)
decoder_3 = Dense(100)(encoder_2)

model = tf.keras.Model(inputs = inputs, outputs = decoder_3)

model.compile(optimizer = 'adam', loss = 'mse')

In [12]:
with tf.device('/GPU:0'):
    model.fit(x[cell_features], x[cell_features], batch_size = 1000, epochs = 200)

Epoch 1/200
24/24 [==============================] - 2s 4ms/step - loss: 2.8981
Epoch 2/200
24/24 [==============================] - 0s 4ms/step - loss: 0.8344
Epoch 3/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5924
Epoch 4/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5500
Epoch 5/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5285
Epoch 6/200
24/24 [==============================] - 0s 4ms/step - loss: 0.5176
Epoch 7/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4930
Epoch 8/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4821
Epoch 9/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4611
Epoch 10/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4408
Epoch 11/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4208
Epoch 12/200
24/24 [==============================] - 0s 4ms/step - loss: 0.4327
Epoch 13/200
24/24 [=================

In [13]:
# take the output of the encoded part
encoder = tf.keras.Model(inputs = inputs, outputs = encoder_2)
encoder.save('encoder_cell_features.h5')

/opt/conda/lib/python3.10/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [8]:
from tensorflow.keras.models import load_model

encoder = load_model('encoder_gene_features.h5')
train_gene_features = encoder.predict(x[gene_features])
test_gene_features = encoder.predict(test[gene_features])

encoder = load_model('encoder_cell_features.h5')
train_cell_features = encoder.predict(x[cell_features])
test_cell_features = encoder.predict(test[cell_features])

2024-03-07 23:49:46.655725: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-03-07 23:49:46.656205: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-03-07 23:49:46.656282: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2024-03-07 23:49:46.656350: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2024-03-07 23:49:46.656417: W tensorflow/c

 39/745 [>.............................] - ETA: 0s  

2024-03-07 23:49:46.954580: W tensorflow/tsl/framework/cpu_allocator_impl.cc:82] Allocation of 147075264 exceeds 10% of free system memory.


125/125 [==============================] - 0s 522us/step


In [9]:
x_1_train = np.hstack((x['cp_type'].values.reshape(-1,1), x['cp_time'].values.reshape(-1, 1), x['cp_dose'].values.reshape(-1, 1),
                        train_gene_features, train_cell_features))

In [16]:
x_1_test = np.hstack((test['cp_type'].values.reshape(-1,1), test['cp_time'].values.reshape(-1, 1), test['cp_dose'].values.reshape(-1, 1),
                        test_gene_features, test_cell_features))

In [17]:
x_1_train.shape, y.shape

((23814, 498), (23814, 206))

## classifer chain - random forest

In [18]:
%%time
classifier = ClassifierChain(classifier = RandomForestClassifier(max_depth = 7, n_estimators = 75),
                                            require_dense=[True, True])
classifier.fit(x_1_train, y)

CPU times: user 59min 26s, sys: 31.2 s, total: 59min 58s
Wall time: 59min 56s


ClassifierChain(classifier=RandomForestClassifier(max_depth=7, n_estimators=75),
                require_dense=[True, True])

In [19]:
pickle.dump((classifier), open('random_forest_model.pkl', 'wb'))

In [20]:
prediction = classifier.predict_proba(x_1_test)

In [21]:
submission = pd.DataFrame(prediction.todense(),columns=[i for i in target_cols])
submission['sig_id'] = test['sig_id']
submission.head()

,5-alpha_reductase_inhibitor,11-beta-hsd1_inhibitor,acat_inhibitor,acetylcholine_receptor_agonist,acetylcholine_receptor_antagonist,acetylcholinesterase_inhibitor,adenosine_receptor_agonist,adenosine_receptor_antagonist,adenylyl_cyclase_activator,adrenergic_receptor_agonist,...,trpv_agonist,trpv_antagonist,tubulin_inhibitor,tyrosine_kinase_inhibitor,ubiquitin_specific_protease_inhibitor,vegfr_inhibitor,vitamin_b,vitamin_d_receptor_agonist,wnt_inhibitor,sig_id
0,0.000014,0.000242,0.000140,0.006538,0.013287,0.014692,0.000974,0.003533,0.000004,0.012739,...,0.000261,0.001121,0.002305,0.001432,0.0,0.001503,0.000221,0.014237,0.000420,id_0004d9e33
1,0.000002,0.000016,0.000159,0.004645,0.009178,0.001411,0.000407,0.002633,0.000008,0.012644,...,0.000037,0.000741,0.002332,0.001428,0.0,0.021163,0.000058,0.000336,0.000229,id_001897cda
2,0.000002,0.000007,0.000248,0.005226,0.010553,0.000904,0.000464,0.015759,0.000000,0.008161,...,0.000043,0.000563,0.003398,0.001641,0.0,0.003239,0.000111,0.000298,0.000269,id_002429b5b
3,0.000181,0.013340,0.000091,0.005656,0.011041,0.006006,0.000966,0.001797,0.000000,0.012086,...,0.000172,0.000387,0.002293,0.001062,0.0,0.001580,0.013578,0.000531,0.000144,id_00276f245
4,0.000038,0.000032,0.000136,0.009333,0.012943,0.001204,0.001105,0.002269,0.000000,0.010177,...,0.000141,0.000866,0.001909,0.001392,0.0,0.002339,0.000190,0.000173,0.000169,id_0027f1083


In [22]:
submission.to_csv("submission.csv",index=False)

![score](score.png)
